<a href="https://colab.research.google.com/github/jonvthvn90/LotteryPredictionAi/blob/main/Artificial_Intelligence_Lottery_Prediction_MegaMillions_PowerBall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# Step 1: Install Required Libraries**


In [1]:
!pip install requests beautifulsoup4


# **Step 2: Fetch Data from the Web Links**
### *Now, a Python script to scrape the historical lottery data from the two links*

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Function to fetch Mega Millions data
def fetch_mega_millions_data():
    url = 'https://www.valottery.com/api/v1/downloadall?gameId=15'  # URL for Mega Millions data
    response = requests.get(url)
    content = response.content.decode('utf-8')  # Decode the byte response

    # Clean and process data
    data = []
    lines = content.splitlines()
    for line in lines:
        if re.match(r"\d{2}/\d{2}/\d{4};", line):  # Match date format
            parts = line.split(';')
            date = parts[0]
            numbers = parts[1].split(',')
            mega_ball = parts[2].split(':')[-1].strip()
            data.append([date] + numbers + [mega_ball])

    # Convert to DataFrame
    columns = ['Date', 'Number1', 'Number2', 'Number3', 'Number4', 'Number5', 'MegaBall']
    df = pd.DataFrame(data, columns=columns)
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

    return df

# Function to fetch Powerball data
def fetch_powerball_data():
    url = 'https://www.valottery.com/api/v1/downloadall?gameId=20'  # URL for Powerball data
    response = requests.get(url)
    content = response.content.decode('utf-8')  # Decode the byte response

    # Clean and process data
    data = []
    lines = content.splitlines()
    for line in lines:
        if re.match(r"\d{2}/\d{2}/\d{4};", line):  # Match date format
            parts = line.split(';')
            date = parts[0]
            numbers = parts[1].split(',')
            powerball = parts[2].split(':')[-1].strip()
            data.append([date] + numbers + [powerball])

    # Convert to DataFrame
    columns = ['Date', 'Number1', 'Number2', 'Number3', 'Number4', 'Number5', 'Powerball']
    df = pd.DataFrame(data, columns=columns)
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

    return df

# Fetch data for Mega Millions and Powerball
mega_millions_data = fetch_mega_millions_data()
powerball_data = fetch_powerball_data()

# Display the fetched data
print("Mega Millions Data:")
print(mega_millions_data.head())

print("\nPowerball Data:")
print(powerball_data.head())


Mega Millions Data:
        Date Number1 Number2 Number3 Number4 Number5 MegaBall
0 2024-10-25      23      26      35      41      43        7
1 2024-10-22       8      43      48      58      60        4
2 2024-10-18       4       9      26      39      58       23
3 2024-10-15      22      34      44      54      62        3
4 2024-10-11       3      10      29      52      57       20

Powerball Data:
        Date Number1 Number2 Number3 Number4 Number5 Powerball
0 2024-10-28      21      27      32      48      67        17
1 2024-10-26       8      12      40      45      51        15
2 2024-10-23       2      15      27      29      39        20
3 2024-10-21       1      25      57      62      64        15
4 2024-10-19       7      16      19      45      64        25


# **Fetching Mega Millions and Powerball Data from APIs**
### *Requests to fetch the data and integrate it into our AI model.*
**Results for 6th Number may be over 25 and 26 Limit. Repurpose that number to inside the the line and replace it with a number under 25 or 26 to their respective games.**

In [ ]:
import requests
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    # Load the data into a pandas dataframe
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        # Check if the row has enough values to unpack
        if row.count(';') >= 2:  # Check for at least 2 semicolons
            date, numbers, extra_ball = row.split(';', 2)  # Limit split to 2
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)
    return df

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]  # Return 5 most frequent numbers

# Supervised Learning (Random Forest)
def random_forest_prediction(data):
    X = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Deep Learning (Neural Network)
def neural_network_prediction(data):
    X = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Genetic Algorithm for evolving predictions
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))  # 1-70 for the numbers

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))  # Fitness based on match with predictions

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]  # Select top 20
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for random probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Combine all techniques to get predictions
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    # Frequency Analysis
    freq_analysis_result = frequency_analysis(data)

    # Random Forest
    rf_result = random_forest_prediction(data)

    # Neural Network
    nn_result = neural_network_prediction(data)

    # Genetic Algorithm Optimization
    ga_result = genetic_algorithm_optimization(freq_analysis_result, data)

    # MCMC Result
    mcmc_result = mcmc_prediction(data)

    # Combine predictions (ensemble approach)
    combined_results = np.concatenate([freq_analysis_result, rf_result, nn_result, ga_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)  # Choose 5 final numbers

    # MegaBall or Powerball prediction
    powerball_prediction = random.choice(data['MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'].values)

    print(f"Numbers: {final_prediction} | {'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'}: {powerball_prediction}")
    return final_prediction, powerball_prediction

# Main function to predict 5 results for MegaMillions and Powerball
def main():
    # Fetch historical data
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    # Generate 5 predictions for Mega Millions
    print("Mega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, 'MegaMillions')

    # Generate 5 predictions for Powerball
    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()


Mega Millions Predictions:
Predicting for: MegaMillions
Numbers: [ 8 10 10  1 17] | MegaBall: 3
Predicting for: MegaMillions
Numbers: [17 19 10  1 31] | MegaBall: 13
Predicting for: MegaMillions
Numbers: [11 31  8 23 46] | MegaBall: 14
Predicting for: MegaMillions
Numbers: [20 50 20 31 10] | MegaBall: 6
Predicting for: MegaMillions
Numbers: [10  1 15 16  8] | MegaBall: 23

Powerball Predictions:
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Numbers: [ 9 21 32  5  5] | Powerball: 4
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Numbers: [39 36 39  4  1] | Powerball: 3
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [15  7 39 36 36] | Powerball: 10
Predicting for: Powerball
Numbers: [36 21 52 36 39] | Powerball: 2
Predicting for: Powerball
Numbers: [11 18 34  4 21] | Powerball: 7


# **Calculate Deciles:**
Compute deciles for each set of drawn numbers (i.e., for each row) and append those decile values as new features.
# **Modify Models to Use Deciles:**
Both the random forest and neural network models will be updated to consider the deciles as additional features.

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    # Load the data into a pandas dataframe
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        # Check if the row has enough values to unpack
        if row.count(';') >= 2:  # Check for at least 2 semicolons
            date, numbers, extra_ball = row.split(';', 2)  # Limit split to 2
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)

    # Add decile calculations
    df = add_deciles(df)

    return df

# Add decile columns to the dataset
def add_deciles(df):
    for col in ['Number1', 'Number2', 'Number3', 'Number4', 'Number5']:
        df[col + '_Decile'] = pd.qcut(df[col], 10, labels=False)
    return df

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]  # Return 5 most frequent numbers

# Supervised Learning (Random Forest) with deciles
def random_forest_prediction(data):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5',
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile']

    X = data[features].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Deep Learning (Neural Network) with deciles
def neural_network_prediction(data):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5',
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile']

    X = data[features].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Genetic Algorithm for evolving predictions
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))  # 1-70 for the numbers

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))  # Fitness based on match with predictions

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]  # Select top 20
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for random probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Combine all techniques to get predictions
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    # Frequency Analysis
    freq_analysis_result = frequency_analysis(data)

    # Random Forest
    rf_result = random_forest_prediction(data)

    # Neural Network
    nn_result = neural_network_prediction(data)

    # Genetic Algorithm Optimization
    ga_result = genetic_algorithm_optimization(freq_analysis_result, data)

    # MCMC Result
    mcmc_result = mcmc_prediction(data)

    # Combine predictions (ensemble approach)
    combined_results = np.concatenate([freq_analysis_result, rf_result, nn_result, ga_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)  # Choose 5 final numbers

    # MegaBall or Powerball prediction
    powerball_prediction = random.choice(data['MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'].values)

    print(f"Numbers: {final_prediction} | {'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'}: {powerball_prediction}")
    return final_prediction, powerball_prediction

# Main function to predict 5 results for MegaMillions and Powerball
def main():
    # Fetch historical data
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    # Generate 5 predictions for Mega Millions
    print("Mega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, 'MegaMillions')

    # Generate 5 predictions for Powerball
    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()


Mega Millions Predictions:
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [17 39  8 17 24] | MegaBall: 18
Predicting for: MegaMillions
Numbers: [64 10 10 44 20] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [ 6 66 31 46 15] | MegaBall: 3
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [31 17 46 20 31] | MegaBall: 52
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [ 6 20 20 17  2] | MegaBall: 7

Powerball Predictions:
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [34 39 23 15  1] | Powerball: 23
Predicting for: Powerball
Numbers: [ 8 20 36 32 21] | Powerball: 4
Predicting for: Powerball
Numbers: [36 59 21  4  4] | Powerball: 13
Predicting for: Powerball
Numbers: [32  4 23 28 10] | Powerball: 28
Predicting for: Powerball
Numbers: [32 23 32  4 33] | Powerball: 5


# **Extract Deciles for the Sixth Number:**
Add a decile column for the sixth number in each game.
# **Modify the Prediction Models:**
 Adjust the random forest and neural network models to also predict the sixth number based on the first five numbers and the sixth number's decile.

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    # Load the data into a pandas dataframe
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        if row.count(';') >= 2:  # Check for at least 2 semicolons
            date, numbers, extra_ball = row.split(';', 2)
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)

    # Add decile calculations
    df = add_deciles(df, ball_type)

    return df

# Add decile columns to the dataset, including the sixth number
def add_deciles(df, ball_type):
    for col in ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]:
        df[col + '_Decile'] = pd.qcut(df[col], 10, labels=False)
    return df

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]  # Return 5 most frequent numbers

# Supervised Learning (Random Forest) with deciles
def random_forest_prediction(data, ball_type):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type,
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile', f'{ball_type}_Decile']

    X = data[features].values
    y = data[ball_type].values.ravel()

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Deep Learning (Neural Network) with deciles
def neural_network_prediction(data, ball_type):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type,
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile', f'{ball_type}_Decile']

    X = data[features].values
    y = data[ball_type].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Genetic Algorithm for evolving predictions
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))  # 1-70 for the numbers

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))  # Fitness based on match with predictions

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]  # Select top 20
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for random probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Combine all techniques to get predictions, including the sixth number
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'

    # Frequency Analysis
    freq_analysis_result = frequency_analysis(data)

    # Random Forest
    rf_result = random_forest_prediction(data, ball_type)

    # Neural Network
    nn_result = neural_network_prediction(data, ball_type)

    # Genetic Algorithm Optimization
    ga_result = genetic_algorithm_optimization(freq_analysis_result, data)

    # MCMC Result
    mcmc_result = mcmc_prediction(data)

    # Combine predictions (ensemble approach)
    combined_results = np.concatenate([freq_analysis_result, rf_result, nn_result, ga_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)  # Choose 5 final numbers

    # Predict the sixth number using the neural network model
    sixth_number_prediction = random.choice(nn_result)

    print(f"Numbers: {final_prediction} | {ball_type}: {sixth_number_prediction}")
    return final_prediction, sixth_number_prediction

# Main function to predict 5 results for MegaMillions and Powerball
def main():
    # Fetch historical data
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    # Generate 5 predictions for Mega Millions
    print("Mega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, 'MegaMillions')

    # Generate 5 predictions for Powerball
    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()



Mega Millions Predictions:
Predicting for: MegaMillions
Numbers: [17 46 50  8  1] | MegaBall: 1
Predicting for: MegaMillions
Numbers: [ 1  1 48 17 20] | MegaBall: 6
Predicting for: MegaMillions
Numbers: [ 6  2 46 20  8] | MegaBall: 6
Predicting for: MegaMillions
Numbers: [46 31 46  1  1] | MegaBall: 6
Predicting for: MegaMillions
Numbers: [ 8 17 17 17 46] | MegaBall: 6

Powerball Predictions:
Predicting for: Powerball
Numbers: [39 32  4 46 54] | Powerball: 24
Predicting for: Powerball
Numbers: [32 24 36 24 43] | Powerball: 24
Predicting for: Powerball
Numbers: [ 2 15 24 32  4] | Powerball: 34
Predicting for: Powerball
Numbers: [32 23  4 34 24] | Powerball: 24
Predicting for: Powerball
Numbers: [39 23  4 21  1] | Powerball: 34


# **Explanation of Updates:**
Frequency Analysis, Genetic Algorithm, MCMC, and Neural Network for First Five Numbers: These methods are each used to predict the first five numbers independently, and results are then combined.
Separate Sixth Number Prediction: The sixth number (MegaBall or Powerball) is treated as a distinct prediction target with its own model. This model uses only the sixth number’s deciles, not the primary five numbers.

Combined Predictions: The final predictions combine results from all methods, with the sixth number derived only from its dedicated prediction model.

This approach ensures that both the first five numbers and the sixth number are handled with specificity, maximizing predictive accuracy across each distinct set of numbers.

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    # Load the data into a pandas dataframe
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        if row.count(';') >= 2:
            date, numbers, extra_ball = row.split(';', 2)
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)

    # Add decile calculations
    df = add_deciles(df, ball_type)

    return df

# Add decile columns to the dataset, focusing on the sixth number
def add_deciles(df, ball_type):
    for col in ['Number1', 'Number2', 'Number3', 'Number4', 'Number5']:
        df[col + '_Decile'] = pd.qcut(df[col], 10, labels=False)
    df[f'{ball_type}_Decile'] = pd.qcut(df[ball_type], 10, labels=False)
    return df

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]

# Genetic Algorithm for evolving predictions
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for random probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Prediction using Deep Learning (Neural Network)
def neural_network_prediction(data):
    X = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5',
              'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile']].values
    y = data['MegaBall' if 'MegaBall' in data.columns else 'Powerball'].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Prediction for the sixth number
def sixth_number_prediction(data, ball_type):
    X = data[[f'{ball_type}_Decile']].values  # Only the decile of the sixth number
    y = data[ball_type].values.ravel()

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)

    sixth_number = model.predict(X[-5:])
    return sixth_number

# Combine all techniques to get predictions, including the sixth number
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'

    # Frequency Analysis
    freq_analysis_result = frequency_analysis(data)

    # Genetic Algorithm
    ga_result = genetic_algorithm_optimization(freq_analysis_result, data)

    # Neural Network
    nn_result = neural_network_prediction(data)

    # MCMC Result
    mcmc_result = mcmc_prediction(data)

    # Sixth Number Prediction using its own model
    sixth_number = sixth_number_prediction(data, ball_type) # changed from sixth_number_prediction to sixth_number

    # Combine predictions
    combined_results = np.concatenate([freq_analysis_result, ga_result, nn_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)  # Choose 5 final numbers

    # Use the sixth number from the specific sixth number model
    sixth_number_pred = random.choice(sixth_number) # changed from sixth_number_prediction to sixth_number_pred

    print(f"Numbers: {final_prediction} | {ball_type}: {sixth_number_pred}") # changed from sixth_number_prediction to sixth_number_pred
    return final_prediction, sixth_number_pred # changed from sixth_number_prediction to sixth_number_pred


# Main function to predict 5 results for MegaMillions and Powerball
def main():
    # Fetch historical data
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    # Generate 5 predictions for Mega Millions
    print("Mega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, 'MegaMillions')

    # Generate 5 predictions for Powerball
    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()


Mega Millions Predictions:
Predicting for: MegaMillions
Numbers: [46 46 10 12  3] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [40 46 31 44  1] | MegaBall: 9
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [17 58 32 10 21] | MegaBall: 4
Predicting for: MegaMillions
Numbers: [34 46 31 30  1] | MegaBall: 3
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [49 12 46 33 46] | MegaBall: 4

Powerball Predictions:
Predicting for: Powerball
Numbers: [ 6 23 36 25  5] | Powerball: 4
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [23 10 32 34 36] | Powerball: 29
Predicting for: Powerball
Numbers: [61 32 39 23 32] | Powerball: 24
Predicting for: Powerball
Numbers: [23 36 21 39  3] | Powerball: 4
Predicting for: Powerball
Numbers: [23  5 20 39  7] | Powerball: 4


# **Explanation of Updates**
Selecting Recent Results: The select_recent_results function filters the dataset to the last six results. This subset is then used across all prediction functions.

Frequency Analysis, Genetic Algorithm, Neural Network, and MCMC on Recent Data: Each prediction method now processes only the recent six results.

Sixth Number Prediction: The sixth number prediction uses a RandomForestClassifier trained on only the recent results for the sixth number, independent of the main five-number set.

Combined Prediction Output: The final prediction aggregates all algorithm outputs from the recent data window for both the five main numbers and the sixth number.

This structure allows the model to prioritize trends within the latest six results, which may highlight recent number patterns or shifts in distribution.

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        if row.count(';') >= 2:
            date, numbers, extra_ball = row.split(';', 2)
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)
    return df

# Select last 6 rows of the data for recent results
def select_recent_results(data, last_n=6):
    return data.tail(last_n)

# Frequency Analysis for recent data
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]

# Genetic Algorithm Optimization for recent data
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for recent data
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Neural Network Prediction for recent data
def neural_network_prediction(data):
    X = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    y = data['MegaBall' if 'MegaBall' in data.columns else 'Powerball'].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Prediction for the sixth number using recent data
def sixth_number_prediction(data, ball_type):
    X = data[[ball_type]].values[:-5]  # Use all but the last 5 rows as features and reshape to 1D
    y = data[ball_type].values[5:]  # Use the last 5 rows as targets and reshape to 1D
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X.reshape(-1, 1), y) # Reshape X to have 1 column

    sixth_number = model.predict(X[-5:].reshape(-1, 1)) # Reshape data for prediction as well to have 1 column
    return sixth_number

# Combine all techniques for predictions based on last 6 results
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    recent_data = select_recent_results(data)
    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'

    # Frequency Analysis on recent data
    freq_analysis_result = frequency_analysis(recent_data)

    # Genetic Algorithm on recent data
    ga_result = genetic_algorithm_optimization(freq_analysis_result, recent_data)

    # Neural Network on recent data
    nn_result = neural_network_prediction(recent_data)

    # MCMC on recent data
    mcmc_result = mcmc_prediction(recent_data)

    # Sixth Number Prediction from recent data
    predicted_sixth_number = sixth_number_prediction(recent_data, ball_type)

    # Combine predictions
    combined_results = np.concatenate([freq_analysis_result, ga_result, nn_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)

    sixth_number_prediction_value = random.choice(predicted_sixth_number)

    print(f"Numbers: {final_prediction} | {ball_type}: {sixth_number_prediction_value}")
    return final_prediction, sixth_number_prediction_value

# Main function to predict 5 results for MegaMillions and Powerball
def main():
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    print("Mega Millions Predictions:")
    for _ in range(6):
        combined_predictions(mega_millions_data, 'MegaMillions')

    print("\nPowerball Predictions:")
    for _ in range(6):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()



Mega Millions Predictions:
Predicting for: MegaMillions
Numbers: [ 8 50  5  6  1] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [47 50 37  5  8] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [ 5  8  8  6 50] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [ 6 29  5 47  8] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [ 5 18 50 48  5] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [ 3  1 47  5 47] | MegaBall: 17

Powerball Predictions:
Predicting for: Powerball
Numbers: [14 36 15 17 59] | Powerball: 24
Predicting for: Powerball
Numbers: [ 8 14 54  1  8] | Powerball: 24
Predicting for: Powerball
Numbers: [22 24 36 59 54] | Powerball: 24
Predicting for: Powerball
Numbers: [ 1 54 54  8 37] | Powerball: 24
Predicting for: Powerball
Numbers: [34 26 54 15 37] | Powerball: 24
Predicting for: Powerball
Numbers: [ 8 38  8 54 54] | Powerball: 24


# Explanation of Key Components:
## Prediction Techniques:
Each prediction technique (frequency analysis, genetic algorithm, neural network, MCMC) works on the last 6 lottery results.
Evolving Predictions:
The evolve_predictions function calculates the accuracy of predictions and updates a new_weights array based on the success rate.

## Looping Predictions:
After each prediction generation, evolve_predictions runs and updates the model’s weights, thus optimizing future predictions.

In a production setting, you would replace placeholders with actual results from lottery drawings to evaluate predictions accurately. Each time new results are available, this algorithm will refine its approach based on accuracy feedback.

In [ ]:
import pandas as pd
import numpy as np
import requests
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch historical data for each lottery
def fetch_data(api_url, ball_type):
    response = requests.get(api_url)
    if response.status_code == 200:
        rows = response.text.splitlines()
        data = []
        for row in rows:
            if row.count(';') >= 2:
                date, numbers, extra_ball = row.split(';', 2)
                num_list = [int(n) for n in numbers.split(',')]
                ball = int(extra_ball.split(':')[-1])
                data.append(num_list + [ball])
        columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
        return pd.DataFrame(data, columns=columns)
    else:
        raise Exception("Failed to fetch data")

# User input for previous prediction outcomes
def get_user_predictions():
    print("Enter the five previous predictions for Mega Millions and Powerball.")
    mega_predictions = [list(map(int, input("Mega Millions Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    mega_ball = [int(input("Mega Millions MegaBall: ")) for _ in range(5)]

    powerball_predictions = [list(map(int, input("Powerball Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    power_ball = [int(input("Powerball Powerball: ")) for _ in range(5)]

    return {
        'MegaMillions': (mega_predictions, mega_ball),
        'Powerball': (powerball_predictions, power_ball)
    }

# Recent results selection
def select_recent_results(data, last_n=50):
    return data.tail(last_n)

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]

# Genetic Algorithm Optimization
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 6
    population = np.random.randint(1, 70, size=(population_size, 5))

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# Neural Network Prediction
def neural_network_prediction(data):
    X = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    y = data[data.columns[-1]].values.ravel()
    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)
    predictions = model.predict(X[-5:])
    return predictions

# MCMC for probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Combined Predictions
def combined_predictions(data, user_predictions, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    recent_data = select_recent_results(data)
    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'

    freq_analysis_result = frequency_analysis(recent_data)
    ga_result = genetic_algorithm_optimization(freq_analysis_result, recent_data)
    nn_result = neural_network_prediction(recent_data)
    mcmc_result = mcmc_prediction(recent_data)

    combined_results = np.concatenate([freq_analysis_result, ga_result, nn_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)
    sixth_number = random.choice(user_predictions[1])

    print(f"Numbers: {final_prediction} | {ball_type}: {sixth_number}")
    return final_prediction, sixth_number

# Main function to run predictions
def main():
    mega_millions_data = fetch_data(MEGA_MILLIONS_API, 'MegaBall')
    powerball_data = fetch_data(POWERBALL_API, 'Powerball')

    user_predictions = get_user_predictions()

    print("\nMega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, user_predictions['MegaMillions'], 'MegaMillions')

    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, user_predictions['Powerball'], 'Powerball')

# Run the main function
main()


Enter the five previous predictions for Mega Millions and Powerball.
Mega Millions Prediction (5 numbers, comma-separated): 5,8,17,18,46
Mega Millions Prediction (5 numbers, comma-separated): 6,88,26,45,46
Mega Millions Prediction (5 numbers, comma-separated): 1,14,17,46,47
Mega Millions Prediction (5 numbers, comma-separated): 17,19,31,46,54
Mega Millions Prediction (5 numbers, comma-separated): 1,4,10,20,46
Mega Millions MegaBall: 17
Mega Millions MegaBall: 17
Mega Millions MegaBall: 8
Mega Millions MegaBall: 8
Mega Millions MegaBall: 8
Powerball Prediction (5 numbers, comma-separated): 1,23,32,36,39
Powerball Prediction (5 numbers, comma-separated): 4,21,31,46,54
Powerball Prediction (5 numbers, comma-separated): 20,23,31,32,37
Powerball Prediction (5 numbers, comma-separated): 15,28,34,39,54
Powerball Prediction (5 numbers, comma-separated): 1,4,23,32,39
Powerball Powerball: 15
Powerball Powerball: 4
Powerball Powerball: 1
Powerball Powerball: 4
Powerball Powerball: 4

Mega Million

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [ 8 32 35  5 35] | MegaBall: 17
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [35  6  4  5 17] | MegaBall: 8
Predicting for: MegaMillions


...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [34 22  4  3 22] | Powerball: 4
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [34 41 52 34 36] | Powerball: 4
Predicting for: Powerball
Numbers: [ 4 37 34 41 52] | Powerball: 4


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Explanation of Changes
## Frequency Analysis per Column:
Modified frequency_analysis_column() to analyze the frequency distribution per column rather than across rows.
## Genetic Algorithm Optimization per Column:
The genetic algorithm now operates per column to optimize predictions based on individual column values.
## Neural Network per Column:
The neural_network_prediction_column() function trains separately on each column to make independent predictions.
Combining Predictions: Each column prediction is aggregated into final_numbers, and the sixth number is selected based on user predictions.

In [ ]:
import pandas as pd
import numpy as np
import requests
import random
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch historical data for each lottery
def fetch_data(api_url, ball_type):
    response = requests.get(api_url)
    if response.status_code == 200:
        rows = response.text.splitlines()
        data = []
        for row in rows:
            if row.count(';') >= 2:
                date, numbers, extra_ball = row.split(';', 2)
                num_list = [int(n) for n in numbers.split(',')]
                ball = int(extra_ball.split(':')[-1])
                data.append(num_list + [ball])
        columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
        return pd.DataFrame(data, columns=columns)
    else:
        raise Exception("Failed to fetch data")

# User input for previous prediction outcomes
def get_user_predictions():
    print("Enter the five previous predictions for Mega Millions and Powerball.")
    mega_predictions = [list(map(int, input("Mega Millions Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    mega_ball = [int(input("Mega Millions MegaBall: ")) for _ in range(5)]

    powerball_predictions = [list(map(int, input("Powerball Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    power_ball = [int(input("Powerball Powerball: ")) for _ in range(5)]

    return {
        'MegaMillions': (mega_predictions, mega_ball),
        'Powerball': (powerball_predictions, power_ball)
    }

# Select recent results for analysis (Adjust Distance from Time)
def select_recent_results(data, last_n=1):
    return data.tail(last_n)

# Frequency Analysis for each column
def frequency_analysis_column(data, column):
    freq = np.bincount(data[column].values, minlength=70)
    return np.argsort(freq)[-5:]

# Decile-based selection for each column
def decile_selection_column(data, column):
    values, counts = np.unique(data[column].values, return_counts=True)
    sorted_indices = np.argsort(-counts)
    decile_threshold = np.percentile(counts, 1)
    top_decile = values[sorted_indices][counts[sorted_indices] >= decile_threshold]
    return np.random.choice(top_decile)

# Genetic Algorithm Optimization for each column
def genetic_algorithm_optimization_column(predictions, column_data):
    population_size = 100
    generations = 6
    population = np.random.randint(1, 70, size=(population_size, 1))

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-5:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(0, 1)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual[0]

# Neural Network Prediction for each column
def neural_network_prediction_column(data, column, ball_type):
    X = data.drop(columns=[ball_type]).values
    y = data[column].values
    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)
    return model.predict(X[-1].reshape(1, -1))[0]

# MCMC Prediction for each column
def mcmc_prediction_column(data, column):
    numbers = data[column].values
    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    return np.random.choice(unique, p=probs)

# Sixth Number Prediction through Algorithms
def sixth_number_prediction(data, ball_column):
    # Frequency Analysis for the sixth number
    numbers = data[ball_column].values
    freq = np.bincount(numbers, minlength=30)
    freq_result = np.argsort(freq)[-1]

    # Genetic Algorithm for the sixth number
    population_size = 100
    generations = 6
    population = np.random.randint(1, 26, size=(population_size,))

    def fitness(individual):
        return np.sum(np.isin(individual, numbers))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            child = random.choice([parent1, parent2])
            next_generation.append(child)
        population = np.array(next_generation)

    ga_result = population[np.argmax(fitness_scores)]

    # Neural Network Prediction for the sixth number
    X = data.iloc[:, :-1].values
    y = data[ball_column].values
    model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
    model.fit(X, y)
    nn_result = model.predict(X[-1].reshape(1, -1))[0]

    # MCMC Prediction for the sixth number
    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    mcmc_result = np.random.choice(unique, p=probs)

    # Combine results for the sixth number
    return random.choice([freq_result, ga_result, nn_result, mcmc_result])

# Combined Predictions
def combined_predictions(data, user_predictions, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    recent_data = select_recent_results(data)
    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'
    max_ball_number = 25 if lottery_type == 'MegaMillions' else 26  # Adjust range for sixth number

    final_numbers = set()
    while len(final_numbers) < 5:
        # Generate prediction for each column
        i = len(final_numbers) + 1
        col_name = f'Number{i}'

        # Frequency Analysis for each column
        freq_result = frequency_analysis_column(recent_data, col_name)

        # Decile-based selection for each column
        decile_result = decile_selection_column(recent_data, col_name)

        # Genetic Algorithm Optimization for each column
        ga_result = genetic_algorithm_optimization_column(freq_result, recent_data[col_name].values)

        # Neural Network Prediction for each column
        nn_result = neural_network_prediction_column(recent_data, col_name, ball_type)

        # MCMC Prediction for each column
        mcmc_result = mcmc_prediction_column(recent_data, col_name)

        # Combining results for each column
        column_result = random.choice([freq_result[0], decile_result, ga_result, nn_result, mcmc_result])
        #Add to roll decile_result

        # Ensure the chosen number is unique and within the required range
        if 1 <= column_result <= 70 and column_result not in final_numbers:
            final_numbers.add(column_result)

    # Convert set to sorted list
    final_numbers = sorted(final_numbers)

    # Predict sixth number using algorithms, within the range of max_ball_number
    sixth_number = sixth_number_prediction(recent_data, ball_type)
    if sixth_number < 1 or sixth_number > max_ball_number:
        sixth_number = random.randint(1, max_ball_number)

    print(f"Numbers: {final_numbers} | {ball_type}: {sixth_number}")
    return final_numbers, sixth_number


# Main function to run predictions
def main():
    mega_millions_data = fetch_data(MEGA_MILLIONS_API, 'MegaBall')
    powerball_data = fetch_data(POWERBALL_API, 'Powerball')

    user_predictions = get_user_predictions()

    print("\nMega Millions Predictions:")
    for _ in range(1):
        combined_predictions(mega_millions_data, user_predictions['MegaMillions'], 'MegaMillions')

    print("\nPowerball Predictions:")
    for _ in range(1):
        combined_predictions(powerball_data, user_predictions['Powerball'], 'Powerball')

# Run the main function
main()



Enter the five previous predictions for Mega Millions and Powerball.
Mega Millions Prediction (5 numbers, comma-separated): 9, 30, 19, 35, 66
Mega Millions Prediction (5 numbers, comma-separated): 70, 47, 19, 14, 52
Mega Millions Prediction (5 numbers, comma-separated): 9, 20, 58, 48, 60
Mega Millions Prediction (5 numbers, comma-separated): 69, 26, 38, 1, 19
Mega Millions Prediction (5 numbers, comma-separated): 3, 70, 39, 42, 17
Mega Millions MegaBall: 16
Mega Millions MegaBall: 6
Mega Millions MegaBall: 7
Mega Millions MegaBall: 15
Mega Millions MegaBall: 1
Powerball Prediction (5 numbers, comma-separated): 40, 24, 28, 65, 63
Powerball Prediction (5 numbers, comma-separated): 23, 02, 16, 63, 04
Powerball Prediction (5 numbers, comma-separated): 17, 47, 55, 50, 40
Powerball Prediction (5 numbers, comma-separated): 58, 1, 28, 51, 13
Powerball Prediction (5 numbers, comma-separated): 54, 12, 33, 36, 28
Powerball Powerball: 20
Powerball Powerball: 13
Powerball Powerball: 6
Powerball Pow

# Trash but keep it because I can edit it.

In [ ]:
import pandas as pd
import numpy as np
import requests
import random
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Set random seed for reproducibility
np.random.seed(42)

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

def fetch_data(api_url, ball_type):
    """
    Fetch lottery data from the specified API URL.
    """
    try:
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()
        rows = response.text.splitlines()
        data = []
        for row in rows[1:]:  # Skip header row
            if row.count(';') >= 2:
                date, numbers, extra_ball = row.split(';', 2)
                num_list = [int(n) for n in numbers.split(',')]
                ball = int(extra_ball.split(':')[-1])
                data.append(num_list + [ball])
        columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
        return pd.DataFrame(data, columns=columns)
    except requests.RequestException as e:
        raise Exception(f"Failed to fetch data from {api_url}: {e}")

def get_user_predictions():
    """
    Collect user predictions for both lotteries.
    """
    try:
        mega_predictions = []
        mega_ball = []
        powerball_predictions = []
        power_ball = []

        for i in range(5):
            mega_pred = input(f"Mega Millions Prediction {i+1} (5 numbers, comma-separated): ").split(',')
            if len(mega_pred) != 5:
                raise ValueError("Each Mega Millions prediction must contain exactly 5 numbers.")
            mega_predictions.append(list(map(int, mega_pred)))

            mega_ball.append(int(input(f"Mega Millions MegaBall {i+1}: ")))

            power_pred = input(f"Powerball Prediction {i+1} (5 numbers, comma-separated): ").split(',')
            if len(power_pred) != 5:
                raise ValueError("Each Powerball prediction must contain exactly 5 numbers.")
            powerball_predictions.append(list(map(int, power_pred)))

            power_ball.append(int(input(f"Powerball Powerball {i+1}: ")))

        return {
            'MegaMillions': (mega_predictions, mega_ball),
            'Powerball': (powerball_predictions, power_ball)
        }
    except ValueError as e:
        print(f"Invalid input: {e}")
        return get_user_predictions()  # Retry

def select_recent_results(data, last_n=4):
    """Select the last n results for analysis."""
    return data.tail(last_n)

def frequency_analysis_column(data, column, max_num):
    """Analyze frequency of numbers in a column."""
    freq = np.bincount(data[column].values, minlength=max_num)
    return np.argsort(freq)[-5:]

def decile_selection_column(data, column, max_num):
    """Select numbers based on decile of frequency."""
    values, counts = np.unique(data[column].values, return_counts=True)
    sorted_indices = np.argsort(-counts)
    decile_threshold = np.percentile(counts, 90)
    top_decile = values[sorted_indices][counts[sorted_indices] >= decile_threshold]
    return np.random.choice(top_decile)

def genetic_algorithm_optimization_column(predictions, column_data, max_num):
    """Genetic algorithm to optimize number selection."""
    population_size, generations = 100, 6
    population = np.random.randint(1, max_num+1, size=(population_size, 1))

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(0, 1)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    return population[np.argmax(fitness_scores)][0]

def neural_network_prediction_column(data, column, ball_type, max_num):
    """Predict numbers using a neural network."""
    X = data.drop(columns=[ball_type]).values
    y = data[column].values
    param_grid = {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
        'activation': ['tanh', 'relu'],
        'max_iter': [500]
    }

    # Check if y has enough samples for 3 splits
    if np.min(np.bincount(y)) >= 3:
        cv = 3  # Use stratified if possible
    else:
        cv = min(2, np.min(np.bincount(y)))  # Reduce to 2 or less if necessary

    grid_search = GridSearchCV(MLPClassifier(), param_grid, cv=cv, n_jobs=-1)
    grid_search.fit(X, y)
    prediction = grid_search.best_estimator_.predict(X[-1].reshape(1, -1))[0]
    return min(max_num, prediction)  # Ensure prediction is within range

def mcmc_prediction_column(data, column, max_num):
    """Use MCMC for number prediction."""
    numbers = data[column].values
    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    return min(max_num, np.random.choice(unique, p=probs))

def sixth_number_prediction(data, ball_column, max_ball):
    """Predict the sixth number using multiple methods."""
    numbers = data[ball_column].values
    freq = np.bincount(numbers, minlength=max_ball)
    freq_result = np.argsort(freq)[-1]

    population_size, generations = 100, 6
    population = np.random.randint(1, max_ball+1, size=(population_size,))

    def fitness(individual):
        return np.sum(np.isin(individual, numbers))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            child = random.choice([parent1, parent2])
            next_generation.append(child)
        population = np.array(next_generation)

    ga_result = population[np.argmax(fitness_scores)]

    X = data.iloc[:, :-1].values
    y = data[ball_column].values
    model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
    model.fit(X, y)
    nn_result = model.predict(X[-1].reshape(1, -1))[0]

    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    mcmc_result = np.random.choice(unique, p=probs)

    return min(max_ball, random.choice([freq_result, ga_result, nn_result, mcmc_result]))

def combined_predictions(data, user_predictions, lottery_type='MegaMillions'):
    """Combine predictions for lottery numbers."""
    print(f"Predicting for: {lottery_type}")
    recent_data = select_recent_results(data)
    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'
    max_ball_number = 25 if lottery_type == 'MegaMillions' else 26
    max_num = 70 if lottery_type == 'MegaMillions' else 60  # Adjust based on lottery type

    final_numbers = set()
    while len(final_numbers) < 5:
        i = len(final_numbers) + 1
        col_name = f'Number{i}'

        freq_result = frequency_analysis_column(recent_data, col_name, max_num)
        decile_result = decile_selection_column(recent_data, col_name, max_num)
        ga_result = genetic_algorithm_optimization_column(freq_result, recent_data[col_name].values, max_num)
        nn_result = neural_network_prediction_column(recent_data, col_name, ball_type, max_num)
        mcmc_result = mcmc_prediction_column(recent_data, col_name, max_num)

        column_result = random.choice([freq_result[0], decile_result, ga_result, nn_result, mcmc_result])
        if 1 <= column_result <= max_num and column_result not in final_numbers:
            final_numbers.add(column_result)

    final_numbers = sorted(final_numbers)
    sixth_number = sixth_number_prediction(recent_data, ball_type, max_ball_number)
    if sixth_number < 1 or sixth_number > max_ball_number:
        sixth_number = random.randint(1, max_ball_number)

    print(f"Numbers: {final_numbers} | {ball_type}: {sixth_number}")
    return final_numbers, sixth_number

def main():
    try:
        mega_millions_data = fetch_data(MEGA_MILLIONS_API, 'MegaBall')
        powerball_data = fetch_data(POWERBALL_API, 'Powerball')

        user_predictions = get_user_predictions()

        print("\nMega Millions Predictions (Max number 70):")
        for _ in range(5):
            combined_predictions(mega_millions_data, user_predictions['MegaMillions'], 'MegaMillions')

        print("\nPowerball Predictions (Max number 60):")
        for _ in range(5):
            combined_predictions(powerball_data, user_predictions['Powerball'], 'Powerball')
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Mega Millions Prediction 1 (5 numbers, comma-separated): 4, 8, 24, 29, 50
Mega Millions MegaBall 1: 17
Powerball Prediction 1 (5 numbers, comma-separated): 5, 24, 29, 37, 48
Powerball Powerball 1: 17
Mega Millions Prediction 2 (5 numbers, comma-separated): 20, 24, 31, 37, 50
Mega Millions MegaBall 2: 6
Powerball Prediction 2 (5 numbers, comma-separated): 5, 11, 18, 29, 37
Powerball Powerball 2: 1
Mega Millions Prediction 3 (5 numbers, comma-separated): 9, 20, 34, 48, 69
Mega Millions MegaBall 3: 17
Powerball Prediction 3 (5 numbers, comma-separated): 5, 18, 22, 37, 52
Powerball Powerball 3: 25
Mega Millions Prediction 4 (5 numbers, comma-separated): 22, 25, 36, 52, 54
Mega Millions MegaBall 4: 24
Powerball Prediction 4 (5 numbers, comma-separated): 14, 18, 20, 37, 52
Powerball Powerball 4: 26
Mega Millions Prediction 5 (5 numbers, comma-separated): 5, 18, 22, 30, 37
Mega Millions MegaBall 5: 24
Powerball Prediction 5 (5 numbers, comma-separated): 14, 22, 36, 40, 52
Powerball Powerball 

New Changes 3/16/225

In [2]:
import pandas as pd
import numpy as np
import requests
import random
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch historical data for each lottery
def fetch_data(api_url, ball_type):
    response = requests.get(api_url)
    if response.status_code == 200:
        rows = response.text.splitlines()
        data = []
        for row in rows:
            if row.count(';') >= 2:
                date, numbers, extra_ball = row.split(';', 2)
                num_list = [int(n) for n in numbers.split(',')]
                ball = int(extra_ball.split(':')[-1])
                data.append(num_list + [ball])
        columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
        return pd.DataFrame(data, columns=columns)
    else:
        raise Exception("Failed to fetch data")

# User input for previous winning numbers (assumed to be actual draws)
def get_user_predictions():
    print("Enter the five most recent winning draws for Mega Millions and Powerball.")
    mega_predictions = [list(map(int, input("Mega Millions Numbers (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    mega_ball = [int(input("Mega Millions MegaBall: ")) for _ in range(5)]

    powerball_predictions = [list(map(int, input("Powerball Numbers (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    power_ball = [int(input("Powerball Powerball: ")) for _ in range(5)]

    return {
        'MegaMillions': (mega_predictions, mega_ball),
        'Powerball': (powerball_predictions, power_ball)
    }

# Frequency Analysis for each column, selecting top 5 most frequent numbers
def frequency_analysis_column(data, column):
    freq = np.bincount(data[column].values, minlength=71)  # Indices 0-70, numbers start at 1
    top_numbers = np.argsort(-freq)[1:6]  # Exclude index 0, take top 5
    return top_numbers

# Decile-based selection for each column
def decile_selection_column(data, column):
    values, counts = np.unique(data[column].values, return_counts=True)
    sorted_indices = np.argsort(-counts)
    decile_threshold = np.percentile(counts, 100)  # Top 1% for more data
    top_decile = values[sorted_indices][counts[sorted_indices] >= decile_threshold]
    return np.random.choice(top_decile)

# Genetic Algorithm Optimization for each column (simplified)
def genetic_algorithm_optimization_column(predictions, column_data):
    population_size = 100
    generations = 6
    population = np.random.randint(1, 70, size=(population_size,))

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-5:]]
        population = np.random.choice(selected, size=population_size)

    return random.choice(selected)

# Neural Network Prediction for each column (using full historical data)
def neural_network_prediction_column(data, column, ball_type):
    X = data.drop(columns=[ball_type]).values
    y = data[column].values
    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)
    return model.predict(X[-1].reshape(1, -1))[0]

# MCMC Prediction for each column
def mcmc_prediction_column(data, column):
    numbers = data[column].values
    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    return np.random.choice(unique, p=probs)

# Sixth Number Prediction
def sixth_number_prediction(data, ball_column, max_ball_number):
    numbers = data[ball_column].values
    freq = np.bincount(numbers, minlength=max_ball_number + 1)
    freq_result = np.argsort(-freq)[1]  # Most frequent, exclude 0

    population_size = 100
    generations = 6
    population = np.random.randint(1, max_ball_number + 1, size=(population_size,))

    def fitness(individual):
        return np.sum(np.isin(individual, numbers))

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        population = np.random.choice(selected, size=population_size)
    ga_result = random.choice(selected)

    X = data.iloc[:, :-1].values
    y = data[ball_column].values
    model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
    model.fit(X, y)
    nn_result = model.predict(X[-1].reshape(1, -1))[0]

    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    mcmc_result = np.random.choice(unique, p=probs)

    last_ball = data[ball_column].iloc[-1]
    ball_candidates = [b for b in [last_ball - 1, last_ball, last_ball + 1] if 1 <= b <= max_ball_number]

    return random.choice([freq_result, ga_result, nn_result, mcmc_result, random.choice(ball_candidates)])

# Combined Predictions with improvements
def combined_predictions(data, user_predictions, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    # Use user-provided recent draws instead of last_n=1 from historical data
    if lottery_type == 'MegaMillions':
        predictions, balls = user_predictions['MegaMillions']
        ball_type = 'MegaBall'
        max_ball_number = 25
    elif lottery_type == 'Powerball':
        predictions, balls = user_predictions['Powerball']
        ball_type = 'Powerball'
        max_ball_number = 26

    recent_data = pd.DataFrame(predictions, columns=['Number1', 'Number2', 'Number3', 'Number4', 'Number5'])
    recent_data[ball_type] = balls

    # Generate candidates for adjacent prediction (±1 from last draw's numbers)
    last_draw = recent_data.iloc[-1]
    last_numbers = last_draw[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    candidates = set()
    for num in last_numbers:
        for d in [-1, 0, 1]:  # Numbers ±1 from last draw
            cand = num + d
            if 1 <= cand <= 70:
                candidates.add(cand)

    final_numbers = set()
    while len(final_numbers) < 5:
        i = len(final_numbers) + 1
        col_name = f'Number{i}'

        freq_result = frequency_analysis_column(recent_data, col_name)
        decile_result = decile_selection_column(recent_data, col_name)
        ga_result = genetic_algorithm_optimization_column(freq_result, recent_data[col_name].values)
        nn_result = neural_network_prediction_column(data, col_name, ball_type)  # Full data for NN
        mcmc_result = mcmc_prediction_column(recent_data, col_name)
        adjacent_result = random.choice(list(candidates))

        # Bias towards adjacent_result since predictions are off by ±1
        column_result = random.choice([adjacent_result, random.choice(freq_result), decile_result, ga_result, nn_result, mcmc_result])

        if 1 <= column_result <= 70 and column_result not in final_numbers:
            final_numbers.add(column_result)

    final_numbers = sorted(final_numbers)

    sixth_number = sixth_number_prediction(recent_data, ball_type, max_ball_number)
    if sixth_number < 1 or sixth_number > max_ball_number:
        sixth_number = random.randint(1, max_ball_number)

    print(f"Numbers: {final_numbers} | {ball_type}: {sixth_number}")
    return final_numbers, sixth_number

# Main function to run predictions
def main():
    mega_millions_data = fetch_data(MEGA_MILLIONS_API, 'MegaBall')
    powerball_data = fetch_data(POWERBALL_API, 'Powerball')

    user_predictions = get_user_predictions()

    print("\nMega Millions Predictions:")
    for _ in range(10):
        combined_predictions(mega_millions_data, user_predictions, 'MegaMillions')

    print("\nPowerball Predictions:")
    for _ in range(10):
        combined_predictions(powerball_data, user_predictions, 'Powerball')

# Run the main function
if __name__ == "__main__":
    main()

Enter the five most recent winning draws for Mega Millions and Powerball.
Mega Millions Numbers (5 numbers, comma-separated): 70, 47, 19, 14, 52
Mega Millions Numbers (5 numbers, comma-separated): 9, 20, 58, 48, 60
Mega Millions Numbers (5 numbers, comma-separated): 69, 26, 38, 1, 19
Mega Millions Numbers (5 numbers, comma-separated): 3, 70, 39, 42, 17
Mega Millions Numbers (5 numbers, comma-separated): 27, 28, 31, 32, 33
Mega Millions MegaBall: 6
Mega Millions MegaBall: 7
Mega Millions MegaBall: 15
Mega Millions MegaBall: 1
Mega Millions MegaBall: 24
Powerball Numbers (5 numbers, comma-separated): 23, 02, 16, 63, 04
Powerball Numbers (5 numbers, comma-separated): 17, 47, 55, 50, 40
Powerball Numbers (5 numbers, comma-separated): 58, 1, 28, 51, 13
Powerball Numbers (5 numbers, comma-separated): 54, 12, 33, 36, 28
Powerball Numbers (5 numbers, comma-separated): 16, 18, 38, 23, 11
Powerball Powerball: 13
Powerball Powerball: 6
Powerball Powerball: 1
Powerball Powerball: 5
Powerball Power

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [np.int64(5), np.int64(14), np.int64(19), np.int64(31), np.int64(52)] | MegaBall: 25
Predicting for: MegaMillions


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [np.int64(1), np.int64(4), np.int64(9), np.int64(28), np.int64(58)] | MegaBall: 7
Predicting for: MegaMillions


...

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [np.int64(13), np.int64(16), np.int64(23), np.int64(51), np.int64(55)] | Powerball: 5
Predicting for: Powerball


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

Numbers: [np.int64(18), np.int64(19), np.int64(28), np.int64(51), np.int64(54)] | Powerball: 9


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## New Roll

In [7]:
import pandas as pd
import numpy as np
import requests
import random
from sklearn.neural_network import MLPRegressor

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch historical data for each lottery
def fetch_data(api_url, ball_type):
    """Fetches historical lottery data from the given API URL."""
    response = requests.get(api_url)
    if response.status_code == 200:
        rows = response.text.splitlines()
        data = []
        for row in rows:
            if row.count(';') >= 2:
                date, numbers, extra_ball = row.split(';', 2)
                num_list = [int(n) for n in numbers.split(',')]
                ball = int(extra_ball.split(':')[-1])
                data.append(num_list + [ball])
        columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
        return pd.DataFrame(data, columns=columns)
    else:
        raise Exception("Failed to fetch data")

# User input for previous prediction outcomes
def get_user_predictions():
    """Collects five previous predictions from the user for both lotteries."""
    print("Enter the five previous predictions for Mega Millions and Powerball.")
    mega_predictions = [list(map(int, input("Mega Millions Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    mega_ball = [int(input("Mega Millions MegaBall: ")) for _ in range(5)]
    powerball_predictions = [list(map(int, input("Powerball Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    power_ball = [int(input("Powerball Powerball: ")) for _ in range(5)]
    return {
        'MegaMillions': (mega_predictions, mega_ball),
        'Powerball': (powerball_predictions, power_ball)
    }

# Select recent results for analysis
def select_recent_results(data, last_n=100):
    """Selects the most recent n draws from the data."""
    return data.tail(last_n)

# Frequency Analysis for main numbers
def frequency_analysis(data):
    """Predicts numbers based on their frequency in historical data."""
    all_numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(all_numbers, minlength=71)[1:]
    top_indices = np.argsort(freq)[-5:]
    return sorted(top_indices + 1)

# Decile-based selection for main numbers
def decile_selection(data):
    """Selects numbers from the top 10% frequency decile."""
    all_numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    values, counts = np.unique(all_numbers, return_counts=True)
    decile_threshold = np.percentile(counts, 90)
    top_decile = values[counts >= decile_threshold]
    return sorted(np.random.choice(top_decile, 5, replace=False))

# Neural Network Prediction for main numbers
def neural_network_prediction(data, ball_type):
    """Uses a neural network regressor to predict the next set of lottery numbers."""
    X = data.drop(columns=[ball_type]).values[:-1]
    y = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values[1:]
    model = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=500)
    if len(X) > 0:
        model.fit(X, y)
        last_draw = data.drop(columns=[ball_type]).values[-1].reshape(1, -1)
        predicted_numbers = model.predict(last_draw)[0]
        predicted_numbers = np.clip(predicted_numbers, 1, 70).astype(int)
        while len(set(predicted_numbers)) < 5:
            new_num = random.randint(1, 70)
            if new_num not in predicted_numbers:
                predicted_numbers = np.append(predicted_numbers, new_num)
                predicted_numbers = np.unique(predicted_numbers)[:5]
        return sorted(predicted_numbers)
    return sorted(np.random.randint(1, 71, 5))

# MCMC Prediction for main numbers
def mcmc_prediction(data):
    """Uses Markov Chain Monte Carlo to sample numbers based on frequency."""
    all_numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    unique, counts = np.unique(all_numbers, return_counts=True)
    probs = counts / counts.sum()
    return sorted(np.random.choice(unique, 5, replace=False, p=probs))

# Sheet Movement Prediction for main numbers
def sheet_movement_prediction(data, max_num=70, num_to_select=5):
    """Simulates number movement across draws to predict numbers."""
    transitions = {i: {} for i in range(1, 6)}
    for i in range(len(data) - 1):
        current = data.iloc[i][['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
        next_draw = data.iloc[i + 1][['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
        for pos, num in enumerate(current, 1):
            if num in next_draw:
                new_pos = np.where(next_draw == num)[0][0] + 1
                transitions[pos].setdefault(num, {}).setdefault(new_pos, 0)
                transitions[pos][num][new_pos] += 1

    last_draw = data.iloc[-1][['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    sheet = list(last_draw)
    selected = set()

    for _ in range(5):
        sheet = sheet[1:] + [sheet[0]]  # Left shift
        if random.random() < 0.5:
            sheet = [sheet[-1]] + sheet[:-1]  # Right shift
        center_num = sheet[2]
        if center_num not in selected and random.random() < 0.7:
            selected.add(center_num)
        else:
            current_pos = random.randint(1, 5)
            num = sheet[current_pos - 1]
            if num in transitions[current_pos] and transitions[current_pos][num]:
                next_pos_probs = transitions[current_pos][num]
                total = sum(next_pos_probs.values())
                probs = {pos: count / total for pos, count in next_pos_probs.items()}
                next_pos = random.choices(list(probs.keys()), weights=list(probs.values()), k=1)[0]
                next_num = sheet[next_pos - 1]
                if next_num not in selected:
                    selected.add(next_num)
            else:
                available = [n for n in sheet if n not in selected]
                if available:
                    selected.add(random.choice(available))

        if len(selected) >= num_to_select:
            break

    while len(selected) < num_to_select:
        new_num = random.randint(1, max_num)
        if new_num not in selected:
            selected.add(new_num)

    return sorted(list(selected)[:num_to_select])

# Genetic Algorithm Optimization for main numbers (Last Algorithm)
def genetic_algorithm_optimization(predictions):
    """Optimizes number selection using a genetic algorithm based on user predictions."""
    population_size = 100
    generations = 50
    max_num = 71
    predictions_flat = set(np.array(predictions).flatten())
    population = [np.random.choice(range(1, max_num), 5, replace=False) for _ in range(population_size)]

    def fitness(individual):
        return sum(num in predictions_flat for num in individual)

    for _ in range(generations):
        fitness_scores = [fitness(ind) for ind in population]
        selected = [population[i] for i in np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = random.sample(selected, 2)
            crossover_point = random.randint(1, 4)
            child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child = np.unique(child)
            while len(child) < 5:
                new_num = random.randint(1, max_num - 1)
                if new_num not in child:
                    child = np.append(child, new_num)
                    child = np.unique(child)
            if random.random() < 0.1:
                idx = random.randint(0, 4)
                new_num = random.randint(1, max_num - 1)
                while new_num in child:
                    new_num = random.randint(1, max_num - 1)
                child[idx] = new_num
            next_generation.append(sorted(child[:5]))
        population = next_generation

    best_idx = np.argmax([fitness(ind) for ind in population])
    return sorted(population[best_idx])

# Sixth Number Prediction (Fixed to throttle high values)
def sixth_number_prediction(data, ball_type, max_ball_number, prev_five_numbers=None):
    """Predicts the sixth number (MegaBall or Powerball) using multiple methods, throttled to max_ball_number."""
    numbers = data[ball_type].values
    # Frequency Analysis
    freq = np.bincount(numbers, minlength=max_ball_number + 1)[1:]
    freq_result = np.argmax(freq) + 1

    # Genetic Algorithm
    population_size = 100
    generations = 20
    population = np.random.randint(1, max_ball_number + 1, size=population_size)

    def fitness(individual):
        return 1 if individual in numbers else 0

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = random.sample(list(selected), 2)
            child = random.choice([parent1, parent2])
            if random.random() < 0.1:
                child = random.randint(1, max_ball_number)
            next_generation.append(child)
        population = np.array(next_generation)
    ga_result = population[np.argmax([fitness(ind) for ind in population])]

    # MCMC Prediction
    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    mcmc_result = np.random.choice(unique, p=probs)

    # Sheet Movement (Ensure max_num is respected)
    if prev_five_numbers:
        temp_df = pd.DataFrame([prev_five_numbers], columns=['Number1', 'Number2', 'Number3', 'Number4', 'Number5'])
        sheet_result = sheet_movement_prediction(temp_df, max_num=max_ball_number, num_to_select=1)[0]
    else:
        sheet_result = random.randint(1, max_ball_number)

    # Throttle all predictions to ensure they don’t exceed max_ball_number
    candidates = [freq_result, ga_result, mcmc_result, sheet_result]
    throttled_candidates = [min(max(1, num), max_ball_number) for num in candidates]
    return random.choice(throttled_candidates)

# Combined Predictions
def combined_predictions(data, user_predictions, lottery_type='MegaMillions'):
    """Generates a final prediction by combining multiple methods, with Genetic Algorithm last."""
    print(f"Predicting for: {lottery_type}")
    recent_data = select_recent_results(data, last_n=100)
    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'
    max_ball_number = 25 if lottery_type == 'MegaMillions' else 26
    max_main_number = 70

    freq_result = frequency_analysis(recent_data)
    decile_result = decile_selection(recent_data)
    nn_result = neural_network_prediction(recent_data, ball_type)
    mcmc_result = mcmc_prediction(recent_data)
    sheet_result = sheet_movement_prediction(recent_data, max_num=max_main_number, num_to_select=5)
    ga_result = genetic_algorithm_optimization(user_predictions[0])  # Genetic Algorithm last

    final_numbers = random.choice([freq_result, decile_result, nn_result, mcmc_result, sheet_result, ga_result])
    sixth_number = sixth_number_prediction(recent_data, ball_type, max_ball_number, prev_five_numbers=final_numbers)

    print(f"Numbers: {final_numbers} | {ball_type}: {sixth_number}")
    return final_numbers, sixth_number

# Main function to run predictions
def main():
    """Runs the lottery prediction program."""
    mega_millions_data = fetch_data(MEGA_MILLIONS_API, 'MegaBall')
    powerball_data = fetch_data(POWERBALL_API, 'Powerball')
    user_predictions = get_user_predictions()

    print("\nMega Millions Predictions:")
    for _ in range(10):
      combined_predictions(mega_millions_data, user_predictions['MegaMillions'], 'MegaMillions')

    print("\nPowerball Predictions:")
    for _ in range(10):
      combined_predictions(powerball_data, user_predictions['Powerball'], 'Powerball')

if __name__ == "__main__":
    main()

Enter the five previous predictions for Mega Millions and Powerball.
Mega Millions Prediction (5 numbers, comma-separated): 70, 47, 19, 14, 52
Mega Millions Prediction (5 numbers, comma-separated): 9, 20, 58, 48, 60
Mega Millions Prediction (5 numbers, comma-separated): 69, 26, 38, 1, 19
Mega Millions Prediction (5 numbers, comma-separated): 3, 70, 39, 42, 17
Mega Millions Prediction (5 numbers, comma-separated): 27, 28, 31, 32, 33
Mega Millions MegaBall: 6
Mega Millions MegaBall: 7
Mega Millions MegaBall: 15
Mega Millions MegaBall: 1
Mega Millions MegaBall: 24
Powerball Prediction (5 numbers, comma-separated): 23, 02, 16, 63, 04
Powerball Prediction (5 numbers, comma-separated): 17, 47, 55, 50, 40
Powerball Prediction (5 numbers, comma-separated): 58, 1, 28, 51, 13
Powerball Prediction (5 numbers, comma-separated): 54, 12, 33, 36, 28
Powerball Prediction (5 numbers, comma-separated): 16, 18, 38, 23, 11
Powerball Powerball: 13
Powerball Powerball: 6
Powerball Powerball: 1
Powerball Pow

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [np.int64(4), np.int64(8), np.int64(35), np.int64(42), np.int64(47)] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [np.int64(4), np.int64(8), np.int64(35), np.int64(42), np.int64(47)] | MegaBall: 15
Predicting for: MegaMillions
Numbers: [np.int64(5), np.int64(11), np.int64(29), np.int64(47), np.int64(50)] | MegaBall: 13
Predicting for: MegaMillions
Numbers: [np.int64(11), np.int64(17), np.int64(31), np.int64(37), np.int64(54)] | MegaBall: 1
Predicting for: MegaMillions
Numbers: [np.int64(4), np.int64(8), np.int64(35), np.int64(42), np.int64(47)] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [np.int64(9), np.int64(39), np.int64(52), np.int64(58), np.int64(70)] | MegaBall: 25
Predicting for: MegaMillions
Numbers: [np.int64(1), np.int64(8), np.int64(14), np.int64(18), np.int64(24)] | MegaBall: 16
Predicting for: MegaMillions
Numbers: [np.int64(4), np.int64(8), np.int64(35), np.int64(42), np.int64(47)] | MegaBall: 17
Predicting for: MegaMillions


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [np.int64(9), np.int64(17), np.int64(52), np.int64(60), np.int64(69)] | MegaBall: 20

Powerball Predictions:
Predicting for: Powerball


...

Numbers: [np.int64(1), np.int64(13), np.int64(36), np.int64(50), np.int64(63)] | Powerball: 11
Predicting for: Powerball
Numbers: [np.int64(9), np.int64(20), np.int64(31), np.int64(47), np.int64(53)] | Powerball: 10
Predicting for: Powerball
Numbers: [np.int64(10), np.int64(20), np.int64(22), np.int64(37), np.int64(52)] | Powerball: 6
Predicting for: Powerball
Numbers: [np.int64(32), np.int64(37), np.int64(38), np.int64(44), np.int64(47)] | Powerball: 26
Predicting for: Powerball
Numbers: [np.int64(17), np.int64(22), np.int64(36), np.int64(37), np.int64(52)] | Powerball: 15
Predicting for: Powerball


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [np.int64(14), np.int64(45), np.int64(49), np.int64(52), np.int64(59)] | Powerball: 15
Predicting for: Powerball
Numbers: [np.int64(2), np.int64(12), np.int64(54), np.int64(55), np.int64(58)] | Powerball: 9
Predicting for: Powerball


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [np.int64(9), np.int64(12), np.int64(14), np.int64(21), np.int64(25)] | Powerball: 26
Predicting for: Powerball
Numbers: [np.int64(4), np.int64(17), np.int64(40), np.int64(50), np.int64(58)] | Powerball: 15
Predicting for: Powerball
Numbers: [np.int64(10), np.int64(20), np.int64(22), np.int64(37), np.int64(52)] | Powerball: 15


### First new one

In [4]:
import pandas as pd
import numpy as np
import requests
import random
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch historical data for each lottery
def fetch_data(api_url, ball_type):
    response = requests.get(api_url)
    if response.status_code == 200:
        rows = response.text.splitlines()
        data = []
        for row in rows:
            if row.count(';') >= 2:
                date, numbers, extra_ball = row.split(';', 2)
                num_list = [int(n) for n in numbers.split(',')]
                ball = int(extra_ball.split(':')[-1])
                data.append(num_list + [ball])
        columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
        return pd.DataFrame(data, columns=columns)
    else:
        raise Exception("Failed to fetch data")

# User input for previous prediction outcomes
def get_user_predictions():
    print("Enter the five previous predictions for Mega Millions and Powerball.")
    mega_predictions = [list(map(int, input("Mega Millions Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    mega_ball = [int(input("Mega Millions MegaBall: ")) for _ in range(5)]

    powerball_predictions = [list(map(int, input("Powerball Prediction (5 numbers, comma-separated): ").split(','))) for _ in range(5)]
    power_ball = [int(input("Powerball Powerball: ")) for _ in range(5)]

    return {
        'MegaMillions': (mega_predictions, mega_ball),
        'Powerball': (powerball_predictions, power_ball)
    }

# Select recent results for analysis
def select_recent_results(data, last_n=100):
    return data.tail(last_n)

# Frequency Analysis for full set
def frequency_analysis(data):
    all_numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(all_numbers, minlength=71)[1:]  # Exclude 0, range 1-70
    return sorted(np.argsort(freq)[-5:]) + 1  # Top 5 numbers, adjust for 1-based indexing

# Decile-based selection for full set
def decile_selection(data):
    all_numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    values, counts = np.unique(all_numbers, return_counts=True)
    decile_threshold = np.percentile(counts, 90)  # Top 10%
    top_decile = values[counts >= decile_threshold]
    return sorted(np.random.choice(top_decile, 5, replace=False))

# Genetic Algorithm Optimization for full set
def genetic_algorithm_optimization(predictions):
    population_size = 100
    generations = 50
    max_num = 71  # 1-70
    predictions_flat = set(np.array(predictions).flatten())

    # Initialize population with unique numbers
    population = [np.random.choice(range(1, max_num), 5, replace=False) for _ in range(population_size)]

    def fitness(individual):
        return sum(num in predictions_flat for num in individual)

    for _ in range(generations):
        fitness_scores = [fitness(ind) for ind in population]
        selected = [population[i] for i in np.argsort(fitness_scores)[-20:]]  # Top 20
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = random.sample(selected, 2)
            crossover_point = random.randint(1, 4)
            child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child = np.unique(child)  # Ensure uniqueness
            while len(child) < 5:  # Fill if duplicates reduced length
                new_num = random.randint(1, max_num - 1)
                if new_num not in child:
                    child = np.append(child, new_num)
                    child = np.unique(child)
            if random.random() < 0.1:  # 10% mutation chance
                idx = random.randint(0, 4)
                new_num = random.randint(1, max_num - 1)
                while new_num in child:
                    new_num = random.randint(1, max_num - 1)
                child[idx] = new_num
            next_generation.append(sorted(child[:5]))
        population = next_generation

    best_idx = np.argmax([fitness(ind) for ind in population])
    return sorted(population[best_idx])

# Neural Network Prediction (simplified, column-based still due to complexity)
def neural_network_prediction(data, ball_type):
    X = data.drop(columns=[ball_type]).values[:-1]  # All but last row as features
    y = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].iloc[-1].values  # Last row as target
    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    if len(X) > 0:  # Ensure there's enough data
        model.fit(X, y)
        pred = model.predict(X[-1].reshape(1, -1))[0]
        while len(set(pred)) < 5:  # Ensure uniqueness
            pred = np.append(pred, random.randint(1, 70))
            pred = np.unique(pred)[:5]
        return sorted(pred)
    return sorted(np.random.randint(1, 71, 5))  # Fallback

# MCMC Prediction for full set
def mcmc_prediction(data):
    all_numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    unique, counts = np.unique(all_numbers, return_counts=True)
    probs = counts / counts.sum()
    return sorted(np.random.choice(unique, 5, replace=False, p=probs))

# Sixth Number Prediction
def sixth_number_prediction(data, ball_type, max_ball_number):
    numbers = data[ball_type].values
    # Frequency Analysis
    freq = np.bincount(numbers, minlength=max_ball_number + 1)[1:]  # Exclude 0
    freq_result = np.argmax(freq) + 1

    # Genetic Algorithm
    population_size = 100
    generations = 20
    population = np.random.randint(1, max_ball_number + 1, size=population_size)

    def fitness(individual):
        return 1 if individual in numbers else 0

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = random.sample(list(selected), 2)
            child = random.choice([parent1, parent2])
            if random.random() < 0.1:  # 10% mutation
                child = random.randint(1, max_ball_number)
            next_generation.append(child)
        population = np.array(next_generation)
    ga_result = population[np.argmax([fitness(ind) for ind in population])]

    # MCMC Prediction
    unique, counts = np.unique(numbers, return_counts=True)
    probs = counts / counts.sum()
    mcmc_result = np.random.choice(unique, p=probs)

    return random.choice([freq_result, ga_result, mcmc_result])

# Combined Predictions
def combined_predictions(data, user_predictions, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")
    recent_data = select_recent_results(data, last_n=100)
    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'
    max_ball_number = 25 if lottery_type == 'MegaMillions' else 26

    # Generate predictions using all methods
    freq_result = frequency_analysis(recent_data)
    decile_result = decile_selection(recent_data)
    ga_result = genetic_algorithm_optimization(user_predictions[0])  # Use main numbers
    nn_result = neural_network_prediction(recent_data, ball_type)
    mcmc_result = mcmc_prediction(recent_data)

    # Randomly choose one set of five numbers
    final_numbers = random.choice([freq_result, decile_result, ga_result, nn_result, mcmc_result])

    # Predict sixth number
    sixth_number = sixth_number_prediction(recent_data, ball_type, max_ball_number)

    print(f"Numbers: {final_numbers} | {ball_type}: {sixth_number}")
    return final_numbers, sixth_number

# Main function to run predictions
def main():
    mega_millions_data = fetch_data(MEGA_MILLIONS_API, 'MegaBall')
    powerball_data = fetch_data(POWERBALL_API, 'Powerball')

    user_predictions = get_user_predictions()

    print("\nMega Millions Predictions:")
    combined_predictions(mega_millions_data, user_predictions['MegaMillions'], 'MegaMillions')

    print("\nPowerball Predictions:")
    combined_predictions(powerball_data, user_predictions['Powerball'], 'Powerball')

# Run the main function
if __name__ == "__main__":
    main()

Enter the five previous predictions for Mega Millions and Powerball.
Mega Millions Prediction (5 numbers, comma-separated): 70, 47, 19, 14, 52
Mega Millions Prediction (5 numbers, comma-separated): 9, 20, 58, 48, 60
Mega Millions Prediction (5 numbers, comma-separated): 69, 26, 38, 1, 19
Mega Millions Prediction (5 numbers, comma-separated): 3, 70, 39, 42, 17
Mega Millions Prediction (5 numbers, comma-separated): 27, 28, 31, 32, 33
Mega Millions MegaBall: 6
Mega Millions MegaBall: 7
Mega Millions MegaBall: 15
Mega Millions MegaBall: 1
Mega Millions MegaBall: 24
Powerball Prediction (5 numbers, comma-separated): 23, 02, 16, 63, 04
Powerball Prediction (5 numbers, comma-separated): 17, 47, 55, 50, 40
Powerball Prediction (5 numbers, comma-separated): 58, 1, 28, 51, 13
Powerball Prediction (5 numbers, comma-separated): 54, 12, 33, 36, 28
Powerball Prediction (5 numbers, comma-separated): 16, 18, 38, 23, 11
Powerball Powerball: 13
Powerball Powerball: 6
Powerball Powerball: 1
Powerball Pow

TypeError: can only concatenate list (not "int") to list